## Zone aggregation

1. At first, you import all the necessary packages and files. 

In [ ]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import geopandas as gpd
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

    Short overview on the next steps given below!

Please be aware that the numbering starts at 2 to be compliant with the notebook numbering.

In [ ]:
# OVERVIEW:
# Step 2: Relabel the zonenumbers to our actual numbers (such that zone 3 has id 3, not id 7 for example)
# Step 3: Split the shapefile into the different regions that will experience different degrees of aggregations
# Step 4: Clustering + retrieving for each original zone to which cluster it belongs
# Step 5: Update the centroids and the OD-matrix

2. Relabel the zonenumbers to the actual numbers (such that zone 3 has id 3, not id 7 for example)

In [ ]:
# STEP 2
city = "BRUSSEL"
radius = "40"

path_orig_shapefile = f"data_map/STA/shapefiles/{city}_{radius}_10/{city}_{radius}_10.shp"
qgis_path = f"data_map/QGIS/{city}_{radius}_10.shp"
shapefile = gpd.read_file(path_orig_shapefile)
shapefile["ZONENUMMER"] = list(range(1,len(shapefile)+1))
shapefile.to_file(qgis_path)

3. This step is done in QGIS. Split the shapefile into the different regions that will experience different degrees of aggregations, in the example case this is chosen to be based on different radiusses.

In [ ]:
# STEP 3: QGIS
'''
The main idea here is to determine the different areas around the area of interest that will experience a different degree of aggregation.
Here, it is decided that:
    * a radius of 10km around Brussels remains unclustered
    * a radius from 10 to 20 km is aggregated into somewhat larger zones
    * a radius from 20 to 40 km is aggregated even more into large zones
For this, the entire region is divided into bands around the center.

Step-by-step guide for QGIS:
    1. Find centroid of entire region of impact 
        OUR CASE: 
            toll_optimization/data_map/QGIS/center.shp

    2. Create circle with 45km radius around centroid. 45km instead of the actual radius of 40km of the area to make sure the most outer zones fall into the created circle.

    3. Split circle into rings with radiuses of 0-10km (circle to be exact), 10-20km and 20-40km. 
        e.g. by creating smaller circles and taking differences of the right combinations of circles.
        OUR CASE:
            toll_optimization/data_map/QGIS/radius_10.shp
            toll_optimization/data_map/QGIS/10-20.shp
            toll_optimization/data_map/QGIS/20-40.shp

    4. For each ring, select the zones of the entire region that are located within the ring. 
        Hint: use the "select within" plugin. There the location is unambiguously decided on the centroid of the zone
        OUR CASE:
            toll_optimization/data_map/QGIS/BRUSSEL_40_10_0-10.shp
            toll_optimization/data_map/QGIS/BRUSSEL_40_10_10-20.shp
            toll_optimization/data_map/QGIS/BRUSSEL_40_10_20-40.shp
'''

4. Clustering step and afterwards also retrieving for each original zone to which cluster it belongs.

In [ ]:
# STEP 4: clustering

# settings
radius1 = "0-10"
radius2 = "10-20"
radius3 = "20-40"
# groupSize1 = 1 not needed
groupSize2 = 4
groupSize3 = 16

# TODO make code more waterproof (naming of input and outputfile)
def aggregate_zones(city = city, radius = radius2, idealNbZonesPerCluster = groupSize2):
    '''
    original shapefile will change: 'cluster' column containing the cluster to which each zone belongs
    new shapefill will be created: shapefile consisting of aggregated zones

    Note:
    code not waterproof: always looks in folder "QGIS"
    '''
    inputFile = f"data_map/QGIS/{city}_40_10_{radius}.shp"
    outputFile = f"data_map/QGIS/{city}_40_10_{radius}_knn.shp"
    shapefile = gpd.read_file(inputFile)

    points = np.array(shapefile.centroid.apply(lambda p: [p.x, p.y]).tolist())

    k =  int(np.ceil(len(shapefile) / idealNbZonesPerCluster))  # number of clusters to create
    print(f'Aggregation of {len(shapefile)} zones into {k} clusters.')
    kmeans = KMeans(n_clusters=k, random_state=0).fit(points) 

    shapefile["cluster"] = kmeans.labels_
    aggregated = shapefile.dissolve(by="cluster").reset_index()
    basicStatisticsZoneSizes = shapefile.groupby('cluster').size().describe() # contains
    print(f"Basic statistics of zone aggregation:\n{basicStatisticsZoneSizes}\n")

    # Create a new attribute column to store which elements each cluster contains
    aggregated["elements_1"] = ""
    aggregated["elements_2"] = ""

    # Retrieve which zones are clustered into which cluster
    cluster_groups = shapefile.groupby("cluster")
    cluster_elements = {}
    for cluster_id, group in cluster_groups:
        element_ids = group["ZONENUMMER"].tolist()
        cluster_elements[cluster_id] = element_ids

    # Update the attribute column with the zonenummers for each cluster. Since the attribute columns are limited in size, 
    # a second column is used if more than 40 zones are clustered into one aggregated zone. 
    for index, row in aggregated.iterrows():
        cluster_id = row["cluster"]
        element_ids = cluster_elements.get(cluster_id, [])
        if len(element_ids) > 40:
            element_ids1 = element_ids[0:40]
            element_ids2 = element_ids[40:]
            aggregated.at[index, "elements_1"] = ", ".join(str(e) for e in element_ids1)
            aggregated.at[index, "elements_2"] = ", ".join(str(e) for e in element_ids2)
        else: 
            aggregated.at[index, "elements_1"] = ", ".join(str(e) for e in element_ids)
    
    # Save the new shapefile to a file & update the original file
    shapefile.to_file(inputFile)
    aggregated.to_file(outputFile)

    # Return aggregated shapefile
    return aggregated

agg_shapefile1 = aggregate_zones()
agg_shapefile2 = aggregate_zones(radius = radius3, idealNbZonesPerCluster = groupSize3)

5. Update the centroids and the OD-matrix

In [ ]:
# STEP 5: Update centroids & OD. First we do some processing to make things easier. 

city = "BRUSSEL"
radius1 = "0-10"
radius2 = "10-20"
radius3 = "20-40"

# Quick fix: add 'cluster' column to unclustered 0-10 radius such that later we have a cluster value for all zones in radius 0-40
# no k-nearest neighbour is ran on the smallest radius, but just for naming convention some processing is done 
shapefile0 = gpd.read_file(f'data_map/QGIS/{city}_40_10_{radius1}.shp')
shapefile0['cluster'] = range(shapefile0.shape[0])
shapefile0['elements_1'] = shapefile0['ZONENUMMER']
shapefile0.to_file(f'data_map/QGIS/{city}_40_10_{radius1}_knn.shp') 

# Concatenate shapefiles
shapefile1 = gpd.read_file(f'data_map/QGIS/{city}_40_10_{radius1}_knn.shp')
shapefile2 = gpd.read_file(f'data_map/QGIS/{city}_40_10_{radius2}_knn.shp')
shapefile3 = gpd.read_file(f'data_map/QGIS/{city}_40_10_{radius3}_knn.shp')

# add scalar to cluster numbers before concatenating shapefiles into combined shapefile (otherwise we start counting from 0 multiple times in the combined shapefile)
shapefile2['cluster'] = shapefile2['cluster'] + (shapefile1.cluster.max()+1)
shapefile3['cluster'] = shapefile3['cluster'] + (shapefile2.cluster.max()+1)

# Save combined shapefile
combined_shapefile = gpd.GeoDataFrame(pd.concat([shapefile1, shapefile2, shapefile3]))
combined_shapefile.to_file(f'data_map/QGIS/{city}_40_10_aggr_comb.shp')

`The final zoning shapefile, containing the aggregated shapefiles all combined, is thus available at: toll_optimization/data_map/QGIS/BRUSSEL_40_10_aggr_comb.shp`

6. Retrieve original OD matrix, centroids and aggregated shapefile.

In [ ]:
original_od_matrix = pd.read_excel(f"data_map/STA/OD_matrix/{city}_40_9_.xlsx") # TODO define dynamically
combined_shapefile = gpd.read_file(f'data_map/QGIS/{city}_40_10_aggr_comb.shp')
original_shapefile = gpd.read_file(f'data_map/QGIS/{city}_40_10.shp')

In [ ]:
# Initialize new OD
nbAggZones = len(combined_shapefile)
aggregated_od_matrix = np.zeros((nbAggZones, nbAggZones))

# Retrieve combined zones of each cluster 
zones_per_cluster = []
for index, row in combined_shapefile.iterrows():
    zones_cluster_str = row['elements_1']
    zones_cluster = [int(e) for e in zones_cluster_str.split(",")]
    if len(zones_cluster) >= 40:
        zones_cluster_str = row['elements_2']
        more_zones = [int(e) for e in zones_cluster_str.split(",")]
        for zone in more_zones:
            zones_cluster.append(zone)
    zones_per_cluster.append(zones_cluster) 

In [ ]:
# Modified OD matrix
# Outer loop
for index_i, row_i in combined_shapefile.iterrows():
    # Retrieve all the zones belonging to cluster i    
    zones_cluster_i = zones_per_cluster[index_i]
    # Inner loop
    for index_j, row_j in combined_shapefile.iterrows():
        # Retrieve all the zones belonging to cluster j 
        zones_cluster_j = zones_per_cluster[index_j]
        
        # Calculate OD flows: sum up the individual flows
        aggr_od_flow = 0
        for zone_i in zones_cluster_i:
            for zone_j in zones_cluster_j:
                # VERY IMPORTANT: PANDAS INDEXING SHOULD FIRST SPECIFY COLUMN, AND THEN THE ROW
                # Otherwise we are flipping the direction of the summed flows... 
                aggr_od_flow += original_od_matrix[zone_j-1][zone_i-1]
        aggregated_od_matrix[index_i][index_j] = aggr_od_flow

# Remove zones that have no demand at all (internal demand is ignored) from the shapefile and the OD_matrix. 
indices_zero_rows = np.where((aggregated_od_matrix==0).all(axis=1))[0]
indices_zero_cols = np.where((aggregated_od_matrix==0).all(axis=0))[0]
aggregated_od_matrix = np.delete(np.delete(aggregated_od_matrix, indices_zero_cols, axis=0), indices_zero_rows, axis=1)
aggregated_od_matrix = pd.DataFrame(aggregated_od_matrix)
combined_shapefile.drop(indices_zero_rows, inplace=True) # Remove entries without creating new geodataframe
combined_shapefile["cluster"] = list(range(0,len(combined_shapefile)))

# Check: no rows or columns are completely filled with zeroes. --> Prints should be empty
indices_zero_rows = np.where((aggregated_od_matrix==0).all(axis=1))[0]
indices_zero_cols = np.where((aggregated_od_matrix==0).all(axis=0))[0]
print(indices_zero_cols)
print(indices_zero_rows)

In [ ]:
# Modified centroids
import typing
from shapely.geometry import Polygon
polygons: typing.List[Polygon] = combined_shapefile["geometry"]
cluster_ids = combined_shapefile["cluster"]

points = [polygon.centroid for polygon in polygons]
centroids = []
centroid_x = []
centroid_y = []
for cluster, point in zip(cluster_ids, points):
    centroids.append({'cluster_id': cluster, 'X_LAMB': point.x, 'Y_LAMB': point.y})
    centroid_x.append(point.x)
    centroid_y.append(point.y)
combined_shapefile["X_LAMB"] = centroid_x
combined_shapefile["Y_LAMB"] = centroid_y
centroids_shapefile = gpd.GeoDataFrame(centroids, geometry=points)
centroids_shapefile.crs = {'init': combined_shapefile.crs}

In [ ]:
# Update centroids in the shapefile
combined_shapefile.to_file(f'data_map/QGIS/{city}_40_10_aggr_comb.shp')
# Save the centroids to a shapefile, such that they can be added as a layer
centroids_shapefile.to_file(f'data_map/QGIS/centroids_BRUSSEL_40_aggr_comb.shp')
# Save new OD flows to excel
aggregated_od_matrix.to_excel("data_map/STA/OD_matrix/BRUSSEL_40_9_aggregated.xlsx", index=False) # Index is False removes the index column (which matches the format of the original OD matrix)

    Congratulations, you have aggregated small zones into larger ones and are set to start assignments on bigger networks!

**** 

## Elasticity matrix

An elasticity matrix between all the aggregated zones is constructed. Elasticities value deviate a standard elasticity, found in literature.

The deviations depend on the slices in which zones are situated.

1. Group zones based on the aggregated slices made in QGIS. In this case specifically, there is one central zone and 6 'pie slices' around it. This is to make a difference between the actual zone of interest (slice 0) and all others providing demand.

In [ ]:
combined_shapefile = gpd.read_file(f'data_map/QGIS/BRUSSEL_40_10_aggr_comb.shp')
# split shapefile into pie slices:
slice_0 = gpd.read_file(f'data_map/QGIS/slices/slice_0.shp')
slice_1 = gpd.read_file(f'data_map/QGIS/slices/slice_1.shp')
slice_2 = gpd.read_file(f'data_map/QGIS/slices/slice_2.shp')
slice_3 = gpd.read_file(f'data_map/QGIS/slices/slice_3.shp')
slice_4 = gpd.read_file(f'data_map/QGIS/slices/slice_4.shp')
slice_5 = gpd.read_file(f'data_map/QGIS/slices/slice_5.shp')
slice_6 = gpd.read_file(f'data_map/QGIS/slices/slice_6.shp')
slices = [slice_0, slice_1, slice_2, slice_3, slice_4, slice_5, slice_6]

# initialize slice list
slice_list = [0 for i in range(len(combined_shapefile))]
i = 1

for slice in slices[1:]:
    clusters = slice['cluster'].values
    for cluster in combined_shapefile['cluster']:
        if cluster in clusters:
            print(cluster)
            slice_list[cluster] = i
    i += 1

combined_shapefile['slice'] = slice_list
combined_shapefile.to_file(f'data_map/QGIS/BRUSSEL_40_10_aggr_comb.shp')

2. Initialise elasticity matrix.

In [ ]:
# initialize elasticity value matrix with ones 
standard_elasticity = 1

standard_elasticity_matrix = [[standard_elasticity for i in range(7)] for j in range(7)]
standard_elasticity_matrix

3. Index all values in the elasticity matrix based on relations between slices (elasticities found in literature, these should normally be thoroughly researched for specific use-cases). Be aware that this is at the moment hardcoded, but it can be improved to have a more flexible style later on.

In [ ]:
# indexing: standard_elasticity_matrix[row][column]
# e.g. standard_elasticity_matrix[0][4] means row 0, column 4. Interpretation: from slice 0 to slice 4

to_own = 0.18
to_neighbour = 0.18
to_others = 0.15
to_opposite = 0.15
to_Brussels = 0.22
from_Brussels = 0.22
in_Brussels = 0.25


# from 0 to 0
standard_elasticity_matrix[0][0] *= in_Brussels

# from i to i
# from i to Brussels
# from Brussels to i
for i in range(1,7):
    standard_elasticity_matrix[i][i] *= to_own
    standard_elasticity_matrix[0][i] *= from_Brussels
    standard_elasticity_matrix[i][0] *= to_Brussels


# from 1 to neighbouring slices
standard_elasticity_matrix[1][2] *= to_neighbour
standard_elasticity_matrix[1][6] *= to_neighbour
# from 1 to opposite slice
standard_elasticity_matrix[1][4] *= to_opposite
# from 1 to other slices
standard_elasticity_matrix[1][3] *= to_others
standard_elasticity_matrix[1][5] *= to_others


# from 2 to neighbouring slices
standard_elasticity_matrix[2][1] *= to_neighbour
standard_elasticity_matrix[2][3] *= to_neighbour
# from 2 to opposite slice
standard_elasticity_matrix[2][5] *= to_opposite
# from 2 to other slices
standard_elasticity_matrix[2][4] *= to_others
standard_elasticity_matrix[2][6] *= to_others


# from 3 to neighbouring slices
standard_elasticity_matrix[3][2] *= to_neighbour
standard_elasticity_matrix[3][4] *= to_neighbour
# from 3 to opposite slice
standard_elasticity_matrix[3][6] *= to_opposite
# from 3 to other slices
standard_elasticity_matrix[3][1] *= to_others
standard_elasticity_matrix[3][5] *= to_others


# from 4 to neighbouring slices
standard_elasticity_matrix[4][3] *= to_neighbour
standard_elasticity_matrix[4][5] *= to_neighbour
# from 4 to opposite slice
standard_elasticity_matrix[4][1] *= to_opposite
# from 4 to other slices
standard_elasticity_matrix[4][2] *= to_others
standard_elasticity_matrix[4][6] *= to_others


# from 5 to neighbouring slices
standard_elasticity_matrix[5][4] *= to_neighbour
standard_elasticity_matrix[5][6] *= to_neighbour
# from 5 to opposite slice
standard_elasticity_matrix[5][2] *= to_opposite
# from 5 to other slices
standard_elasticity_matrix[5][1] *= to_others
standard_elasticity_matrix[5][3] *= to_others

# from 6 to neighbouring slices
standard_elasticity_matrix[6][5] *= to_neighbour
standard_elasticity_matrix[6][1] *= to_neighbour
# from 6 to opposite slice
standard_elasticity_matrix[6][3] *= to_opposite
# from 6 to other slices
standard_elasticity_matrix[6][2] *= to_others
standard_elasticity_matrix[6][4] *= to_others

# round results up to 3 
slice_elasticity_matrix =  [[round(standard_elasticity_matrix[i][j], 3) for i in range(7)] for j in range(7)]

slice_elasticity_matrix


4. Make a zone elasticity matrix (600x600) using the slice elasticity matrix (7x7).

In [ ]:
combined_shapefile = gpd.read_file(f'data_map/QGIS/BRUSSEL_40_10_aggr_comb.shp')
zone_elasticity_matrix = np.zeros((len(combined_shapefile), len(combined_shapefile)))

# Loop through all pairs of observations and fill in the matrix with the corresponding slice value
for i in range(len(combined_shapefile)):
    for j in range(len(combined_shapefile)):
        k = combined_shapefile.iloc[i]['slice']
        l = combined_shapefile.iloc[j]['slice']
        zone_elasticity_matrix[i][j] = slice_elasticity_matrix[k][l]

5. Determine the useful path to the elasticity matrix. This is for ease of use later on.

In [ ]:
elasticity_path = f"data_map/STA/elasticity/Brussel_40"
np.savetxt(elasticity_path,zone_elasticity_matrix)

    Congratulations, you now have made your elasticity matrix B as a building block to perform elastic assignments!